## 날짜별 워드 클라우드로 가장 빈도수 높은 단어 도출 (title)
- 이예빈
- 참고: 불용어 사전 (https://www.ranks.nl/stopwords/korean)

In [12]:
import pandas as pd
import numpy as np
import re
from konlpy.tag import Okt
import numpy as np
import pandas as pd
import re
import json
import os
import glob
from konlpy.tag import Okt
import matplotlib.pyplot as plt
import MeCab
from wordcloud import WordCloud
from sklearn.model_selection import train_test_split
import tqdm

In [18]:
try:
    if not(os.path.isdir('../../data/press')):
        os.makedirs(os.path.join('../../data/press'))
                    
except OSError as e:
    if e.errno != errno.EEXIST:
        print("Failed to create directory!")
        raise

In [19]:
# CSV_List로 만드는 함수
def csv2list(filename):
    lists = []
    file = open(filename,'r',encoding='utf-8')
    while True:
        line = file.readline().rstrip('\n')
        if line:
            lists.append(line)
        else:
            break
    return lists

In [20]:
stop_words = csv2list('../../data/stop_words/stop_words.csv')

In [21]:
### 뉴스 제목에서 기호 제거 & 명사 추출 

def title_preprocessing(title, okt):
    title_text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]', ' ', title)
    word_title = okt.nouns(title_text) 
    word_title = [word for word in word_title if not word in stop_words] # 불용어 제거
    
    return word_title

In [31]:
### 전처리된 데이터프레임 by_day를 반환합니다. 
## file: csv파일이 들어있는 경로를 지정해주세요. (+ 파일까지 포함되도록!)

def all_preprocessing(file):
    df_file = pd.read_csv(file, encoding='utf-8')
    
    title = df_file['title']
    date_ = df_file['date']
    df = pd.DataFrame({'title':title, 'date':date_})
    
    okt = Okt()
    clean_text = []
    for title in df_file['title']:
        if type(title)==str:
            clean_text.append(title_preprocessing(title, okt))
        else:
            clean_text.append([])
    df['title'] = clean_text
    
    # date 컬럼 -> datetime으로 설정 
    df['date'] = pd.to_datetime(df['date'])
    by_day = df.groupby(df['date']).sum()

    # 딕셔너리 풀기
    title_text = []
    for i in range(len(by_day['title'])):
        title_text.append(', '.join(by_day['title'][i]))

    by_day['title'] = title_text
    by_day.to_csv('../../data/press/donga _title.csv', encoding='utf-8')
    
    return by_day

In [32]:
all_preprocessing('../../data/news_by_press(2020-01-01~2020-08-31)/동아일보')

,title
date,
2020-01-01,"공무원, 증, 밥값, 결제, 혁신, 준, 정부, 나은, 준비, 민주, 열망, 시대,..."
2020-01-02,"김정은, 전략, 무기, 모라토리엄, 파기, 위협, 전문가, 공직, 배급, 스타트업,..."
2020-01-03,"처지, 끝장, 생존, 게임, 구도, 오른, 총선, 전쟁, 패트, 몸싸움, 여야, 총..."
2020-01-04,"트럼프, 중동, 화약고, 기업, 법, 시행, 땐, 툭하면, 공장, 이낙연, 황교안,..."
2020-01-06,"법무부, 검사, 출신, 검찰, 국장, 기용, 검토, 이란, 시설, 타깃, 트럼프, ..."
...,...
2020-08-27,"의협, 파업, 강행, 정부, 면허, 취소, 압박, 기상청, 차량, 전복, 위험, 서..."
2020-08-28,"단독, 교수, 진도, 파업, 동참, 서울대, 병원, 내과, 서나, 감염, 대통령, ..."
2020-08-29,"코로나, 방역, 정은경, 하루, 감염, 국방, 장관, 서욱, 총장, 집권, 아베, ..."


### 해당 날짜를 입력하면 워드클라우드로 만들어져 그날 많이 나온 핵심 단어 확인 가능

In [10]:
def daily_news_title_wordcloud( date_to_search):
    tmp_df = pd.read_csv('../../data/news_by_press(2020-01-01~2020-08-31)/')
    
    cloud = WordCloud(
      font_path='../../fonts/NanumSquareRoundR.ttf'
    , width=800
    , height=600
    , max_words=100
    ).generate(''.join(tmp_df['title' ][tmp_df['date'] == date_to_search]))

    plt.figure(figsize=(20, 15))
    plt.imshow(cloud)
    plt.axis('off')

In [11]:
daily_news_title_wordcloud('../../data/news_by_press(2020-01-01~2020-08-31)/경향신문','2020-01-01')

KeyError: 'date'